In [1]:
from __future__ import division
from __future__ import print_function

import time
import os

# Train on CPU (hide GPU) due to memory constraints
os.environ['CUDA_VISIBLE_DEVICES'] = ""
import pandas as pd
import tensorflow as tf
import numpy as np
import scipy.sparse as sp
import scipy
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
import numpy as np
import scipy.sparse as sp
from optimizer import OptimizerAE, OptimizerVAE
from input_data import load_data, parse_index_file
from model import GCNModelAE, GCNModelVAE
from preprocessing import preprocess_graph, construct_feed_dict, sparse_to_tuple, mask_test_edges

In [2]:
# Settings
flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_float('learning_rate', 0.01, 'Initial learning rate.')
flags.DEFINE_integer('epochs', 200, 'Number of epochs to train.')
flags.DEFINE_integer('hidden1', 32, 'Number of units in hidden layer 1.')
flags.DEFINE_integer('hidden2', 16, 'Number of units in hidden layer 2.')
flags.DEFINE_float('weight_decay', 0., 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_float('dropout', 0., 'Dropout rate (1 - keep probability).')

flags.DEFINE_string('model', 'gcn_ae', 'Model string.')
flags.DEFINE_string('dataset', 'cora', 'Dataset string.')
flags.DEFINE_integer('features', 1, 'Whether to use features (1) or not (0).')

model_str = FLAGS.model
dataset_str = FLAGS.dataset

In [24]:
X = pd.read_hdf('../../history_small.hdf', key='hist') 
A = pd.read_pickle('../../adjacency_small.pkl') 
A = A[sorted(A.columns)]

In [25]:
X[X==1] = 2 #10 #valid
X[X==0] = 1 #not valid
X[X==-100] = 0 #missing
X = X.astype(float)
np.unique(X)

array([ 0.,  1.,  2.])

In [26]:
A[A<=0.75] = 0

In [28]:
adj = scipy.sparse.csr_matrix(A.values)
features = scipy.sparse.csr_matrix(features)#np.asmatrix(X)

In [29]:
print(features.shape, adj.shape)
print(type(features), type(adj)) 

(878, 290) (878, 878)
<class 'scipy.sparse.csr.csr_matrix'> <class 'scipy.sparse.csr.csr_matrix'>


In [30]:
adj

<878x878 sparse matrix of type '<class 'numpy.float64'>'
	with 393718 stored elements in Compressed Sparse Row format>

In [31]:
features

<878x290 sparse matrix of type '<class 'numpy.float64'>'
	with 8821 stored elements in Compressed Sparse Row format>

In [32]:
# Store original adjacency matrix (without diagonal entries) for later
adj_orig = adj
adj_orig = adj_orig - sp.dia_matrix((adj_orig.diagonal()[np.newaxis, :], [0]), shape=adj_orig.shape)
adj_orig.eliminate_zeros()

In [33]:
adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
adj.eliminate_zeros()
# Check that diag is zero:
assert np.diag(adj.todense()).sum() == 0

adj_triu = sp.triu(adj)
adj_tuple = sparse_to_tuple(adj_triu)
edges = adj_tuple[0]
edges_all = sparse_to_tuple(adj)[0]
num_test = int(np.floor(edges.shape[0] / 50.))
num_val = int(np.floor(edges.shape[0] / 50.))

all_edge_idx = list(range(edges.shape[0]))
np.random.shuffle(all_edge_idx)
val_edge_idx = all_edge_idx[:num_val]
test_edge_idx = all_edge_idx[num_val:(num_val + num_test)]
test_edges = edges[test_edge_idx]
val_edges = edges[val_edge_idx]
train_edges = np.delete(edges, np.hstack([test_edge_idx, val_edge_idx]), axis=0)

In [34]:
def ismember(a, b, tol=5):
    rows_close = np.all(np.round(a - b[:, None], tol) == 0, axis=-1)
    return (np.all(np.any(rows_close, axis=-1), axis=-1) and
            np.all(np.any(rows_close, axis=0), axis=0))

In [35]:
test_edges_false = []
while len(test_edges_false) < len(test_edges):
    idx_i = np.random.randint(0, adj.shape[0])
    idx_j = np.random.randint(0, adj.shape[0])
    if idx_i == idx_j:
        continue
    if ismember([idx_i, idx_j], edges_all):
        continue
    if test_edges_false:
        if ismember([idx_j, idx_i], np.array(test_edges_false)):
            continue
        if ismember([idx_i, idx_j], np.array(test_edges_false)):
            continue
    test_edges_false.append([idx_i, idx_j])
    print('\r%d / %d' % (len(test_edges_false), len(test_edges)), end='')

3928 / 3928

In [36]:
val_edges_false = []
while len(val_edges_false) < len(val_edges):
    idx_i = np.random.randint(0, adj.shape[0])
    idx_j = np.random.randint(0, adj.shape[0])
    if idx_i == idx_j:
        continue
    if ismember([idx_i, idx_j], train_edges):
        continue
    if ismember([idx_j, idx_i], train_edges):
        continue
    if ismember([idx_i, idx_j], val_edges):
        continue
    if ismember([idx_j, idx_i], val_edges):
        continue
    if val_edges_false:
        if ismember([idx_j, idx_i], np.array(val_edges_false)):
            continue
        if ismember([idx_i, idx_j], np.array(val_edges_false)):
            continue
    val_edges_false.append([idx_i, idx_j])
    print('\r%d / %d' % (len(val_edges_false), len(val_edges)), end='')

3928 / 3928

In [37]:
#assert ~ismember(test_edges_false, edges_all)
#assert ~ismember(val_edges_false, edges_all)
#assert ~ismember(val_edges, train_edges)
#assert ~ismember(test_edges, train_edges)
#assert ~ismember(val_edges, test_edges)

In [38]:
data = np.ones(train_edges.shape[0])
# Re-build adj matrix
adj_train = sp.csr_matrix((data, (train_edges[:, 0], train_edges[:, 1])), shape=adj.shape)
adj_train = adj_train + adj_train.T

In [39]:
#adj_train, train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges(adj)
#adj = adj_train

In [40]:
if FLAGS.features == 0:
    features = sp.identity(features.shape[0])  # featureless

In [41]:
# Some preprocessing
adj_norm = preprocess_graph(adj)

In [42]:
# Define placeholders
placeholders = {
    'features': tf.sparse_placeholder(tf.float32),
    'adj': tf.sparse_placeholder(tf.float32),
    'adj_orig': tf.sparse_placeholder(tf.float32),
    'dropout': tf.placeholder_with_default(0., shape=())
}

In [43]:
num_nodes = adj.shape[0]

In [44]:
features = sparse_to_tuple(features.tocoo())
num_features = features[2][1]
features_nonzero = features[1].shape[0]

In [45]:
# Create model
model = None
if model_str == 'gcn_ae':
    model = GCNModelAE(placeholders, num_features, features_nonzero)
elif model_str == 'gcn_vae':
    model = GCNModelVAE(placeholders, num_features, num_nodes, features_nonzero)

pos_weight = float(adj.shape[0] * adj.shape[0] - adj.sum()) / adj.sum()
norm = adj.shape[0] * adj.shape[0] / float((adj.shape[0] * adj.shape[0] - adj.sum()) * 2)

In [46]:
# Optimizer
with tf.name_scope('optimizer'):
    if model_str == 'gcn_ae':
        opt = OptimizerAE(preds=model.reconstructions,
                          labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                      validate_indices=False), [-1]),
                          pos_weight=pos_weight,
                          norm=norm)
    elif model_str == 'gcn_vae':
        opt = OptimizerVAE(preds=model.reconstructions,
                           labels=tf.reshape(tf.sparse_tensor_to_dense(placeholders['adj_orig'],
                                                                       validate_indices=False), [-1]),
                           model=model, num_nodes=num_nodes,
                           pos_weight=pos_weight,
                           norm=norm)

In [47]:
# Initialize session
sess = tf.Session()
sess.run(tf.global_variables_initializer())

cost_val = []
acc_val = []

In [48]:
def get_roc_score(edges_pos, edges_neg, emb=None):
    if emb is None:
        feed_dict.update({placeholders['dropout']: 0})
        emb = sess.run(model.z_mean, feed_dict=feed_dict)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    # Predict on test set of edges
    adj_rec = np.dot(emb, emb.T)
    preds = []
    pos = []
    for e in edges_pos:
        preds.append(sigmoid(adj_rec[e[0], e[1]]))
        pos.append(adj_orig[e[0], e[1]])

    preds_neg = []
    neg = []
    for e in edges_neg:
        preds_neg.append(sigmoid(adj_rec[e[0], e[1]]))
        neg.append(adj_orig[e[0], e[1]])

    preds_all = np.hstack([preds, preds_neg])
    labels_all = np.hstack([np.ones(len(preds)), np.zeros(len(preds))])
    roc_score = roc_auc_score(labels_all, preds_all)
    ap_score = average_precision_score(labels_all, preds_all)

    return roc_score, ap_score

In [49]:
cost_val = []
acc_val = []
val_roc_score = []

adj_label = adj_train + sp.eye(adj_train.shape[0])
adj_label = sparse_to_tuple(adj_label)

# Train model
for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(adj_norm, adj_label, features, placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})
    # Run single weight update
    outs = sess.run([opt.opt_op, opt.cost, opt.accuracy], feed_dict=feed_dict)

    # Compute average loss
    avg_cost = outs[1]
    avg_accuracy = outs[2]

    roc_curr, ap_curr = get_roc_score(val_edges, val_edges_false)
    val_roc_score.append(roc_curr)

    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(avg_cost),
          "train_acc=", "{:.5f}".format(avg_accuracy), "val_roc=", "{:.5f}".format(val_roc_score[-1]),
          "val_ap=", "{:.5f}".format(ap_curr),
          "time=", "{:.5f}".format(time.time() - t))

print("Optimization Finished!")

Epoch: 0001 train_loss= 0.66534 train_acc= 0.49035 val_roc= 0.65255 val_ap= 0.61621 time= 0.69629
Epoch: 0002 train_loss= 0.84849 train_acc= 0.49035 val_roc= 0.65392 val_ap= 0.61654 time= 0.68130
Epoch: 0003 train_loss= 0.65636 train_acc= 0.49035 val_roc= 0.65771 val_ap= 0.61716 time= 0.74666
Epoch: 0004 train_loss= 0.68005 train_acc= 0.49035 val_roc= 0.65709 val_ap= 0.61649 time= 0.58731
Epoch: 0005 train_loss= 0.67681 train_acc= 0.49035 val_roc= 0.65459 val_ap= 0.61586 time= 0.57502
Epoch: 0006 train_loss= 0.65980 train_acc= 0.49035 val_roc= 0.65399 val_ap= 0.61583 time= 0.56953
Epoch: 0007 train_loss= 0.66946 train_acc= 0.49035 val_roc= 0.65410 val_ap= 0.61573 time= 0.57637
Epoch: 0008 train_loss= 0.67308 train_acc= 0.49035 val_roc= 0.65435 val_ap= 0.61569 time= 0.56157
Epoch: 0009 train_loss= 0.66303 train_acc= 0.49035 val_roc= 0.65470 val_ap= 0.61570 time= 0.56931
Epoch: 0010 train_loss= 0.65818 train_acc= 0.49035 val_roc= 0.65497 val_ap= 0.61565 time= 0.68519
Epoch: 0011 train_lo

In [50]:
roc_score, ap_score = get_roc_score(test_edges, test_edges_false)
print('Test ROC score: ' + str(roc_score))
print('Test AP score: ' + str(ap_score))

Test ROC score: 0.678079054602
Test AP score: 0.613249032012
